In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

# Functions

In [2]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [3]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,method
0,0.099594,exp1,Sphere,1.0,NN_HMu
1,0.099960,exp1,Sphere,1.0,NN_HMu
2,0.100126,exp1,Sphere,1.0,NN_HMu
3,0.099228,exp1,Sphere,1.0,NN_HMu
4,0.098991,exp1,Sphere,1.0,NN_HMu


In [5]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])
table2

experiment             exp1                                              exp2  \
freq                   0.5       1.0       4.0       10.0      20.0      0.5    
function   method                                                               
Rastrigin  NN_CwN  0.195993  0.098576  0.024651  0.010280  0.004885  0.198150   
           NN_HMu  0.196709  0.099099  0.024930  0.009736  0.006756  0.194942   
           NN_No   0.197927  0.099414  0.026144  0.009887  0.004953  0.202033   
           NN_RI   0.194784  0.103330  0.024733  0.010041  0.005139  0.193396   
Rosenbrock NN_CwN  0.194145  0.096923  0.024492  0.010212  0.004846  0.194467   
           NN_HMu  0.196627  0.097056  0.025613  0.009825  0.005183  0.196827   
           NN_No   0.198279  0.098928  0.025669  0.009996  0.004865  0.197987   
           NN_RI   0.197705  0.103244  0.024434  0.010132  0.005082  0.194635   
Sphere     NN_CwN  0.192697  0.097401  0.024537  0.009996  0.004836  0.193567   
           NN_HMu  0.197433  0.099553  0.025041  0.009810  0.005233  0.197951   
           NN_No   0.195678  0.097970  0.024792  0.009846  0.004909  0.194553   
           NN_RI   0.198014  0.104334  0.024904  0.009945  0.005132  0.200087   

experiment                                                     exp3            \
freq                   1.0       4.0       10.0      20.0      0.5       1.0    
function   method                                                               
Rastrigin  NN_CwN  0.098478  0.024668  0.009932  0.005023  0.199739  0.099651   
           NN_HMu  0.096852  0.024448  0.009927  0.004923  0.203036  0.103510   
           NN_No   0.097072  0.025578  0.009970  0.004980  0.201550  0.098819   
           NN_RI   0.104143  0.024949  0.009999  0.005264  0.209618  0.104336   
Rosenbrock NN_CwN  0.097875  0.024164  0.009756  0.004900  0.201342  0.100439   
           NN_HMu  0.098326  0.024710  0.009848  0.004895  0.200598  0.100453   
           NN_No   0.097883  0.025244  0.009765  0.004900  0.201156  0.099705   
           NN_RI   0.105164  0.024503  0.010066  0.005040  0.200695  0.106333   
Sphere     NN_CwN  0.097327  0.024467  0.009681  0.004881  0.201939  0.102522   
           NN_HMu  0.098530  0.025262  0.009768  0.004922  0.201628  0.099219   
           NN_No   0.097408  0.025163  0.009761  0.004989  0.199833  0.098538   
           NN_RI   0.104057  0.024655  0.010175  0.005018  0.199542  0.105392   

experiment                                           exp4                      \
freq                   4.0       10.0      20.0      0.5       1.0       4.0    
function   method                                                               
Rastrigin  NN_CwN  0.025686  0.010042  0.005047  0.190784  0.094339  0.024656   
           NN_HMu  0.024645  0.010244  0.004968  0.193125  0.103063  0.023938   
           NN_No   0.024530  0.010082  0.004893  0.194515  0.096451  0.024623   
           NN_RI   0.024923  0.009925  0.005186  0.199437  0.104475  0.024178   
Rosenbrock NN_CwN  0.025596  0.009972  0.004995  0.191860  0.096562  0.024746   
           NN_HMu  0.024713  0.009940  0.005076  0.192772  0.103874  0.024473   
           NN_No   0.024530  0.010163  0.004930  0.193659  0.096469  0.024365   
           NN_RI   0.025195  0.010301  0.005072  0.204566  0.104762  0.024407   
Sphere     NN_CwN  0.025082  0.009974  0.005027  0.188090  0.095376  0.024688   
           NN_HMu  0.024538  0.010047  0.005101  0.192615  0.100112  0.024787   
           NN_No   0.024675  0.010680  0.004922  0.192025  0.096246  0.024298   
           NN_RI   0.025068  0.010108  0.005224  0.200157  0.101689  0.024595   

experiment                             
freq                   10.0      20.0  
function   method                      
Rastrigin  NN_CwN  0.009475  0.004758  
           NN_HMu  0.010027  0.004872  
           NN_No   0.009620  0.004804  
           NN_RI   0.009899  0.004828  
Rosenbrock NN_CwN  0.009781  0.004784  
           NN_HMu  0.010083  0.0

In [6]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp1,exp1,exp1,exp1,exp2,exp2,exp2,exp2,exp2,exp3,exp3,exp3,exp3,exp3,exp4,exp4,exp4,exp4,exp4
freq,,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0
function,method,,,,,,,,,,,,,,,,,,,,
Rastrigin,NN_CwN,0.194,0.097,0.024,0.010,0.005,0.197,0.099,0.025,0.010,0.005,0.200,0.099,0.025,0.010,0.005,0.189,0.094,0.023,0.010,0.005
Rastrigin,NN_HMu,0.192,0.098,0.024,0.010,0.007,0.194,0.097,0.025,0.010,0.005,0.201,0.136,0.025,0.010,0.007,0.192,0.099,0.024,0.010,0.005
Rastrigin,NN_No,0.197,0.099,0.024,0.010,0.005,0.196,0.098,0.025,0.010,0.005,0.201,0.101,0.024,0.010,0.005,0.196,0.095,0.024,0.010,0.005
Rastrigin,NN_RI,0.198,0.098,0.025,0.010,0.005,0.197,0.099,0.024,0.010,0.005,0.203,0.138,0.025,0.010,0.005,0.194,0.098,0.025,0.010,0.005
Rosenbrock,NN_CwN,0.196,0.098,0.024,0.010,0.005,0.192,0.097,0.024,0.010,0.005,0.200,0.100,0.025,0.010,0.005,0.193,0.095,0.023,0.009,0.005
Rosenbrock,NN_HMu,0.193,0.097,0.024,0.010,0.005,0.195,0.097,0.025,0.010,0.